## Loading all libraries, setting data path

In [ ]:
import os
from glob import glob
import pandas as pd
import numpy as np
from pycaret.time_series import *
base_path = os.getcwd()
data_path = glob(os.path.join(base_path,"input","*.xlsx"))[0]
print (data_path)

# Loading Excel Data

In [2]:
excel_data = pd.ExcelFile(data_path)

## Extracting train and test data from respective sheets

In [3]:
excel_data.sheet_names
train_data = excel_data.parse("Training Dataset")
test_data = excel_data.parse("Test Dataset")
# Fields
# 'ProductType', 'Manufacturer', 'Area Code', 'Sourcing Channel','Product Size',
# 'Product Type', 'Month of Sourcing', 'Sourcing Cost'

## Preparing the label by combining all the ABCDE columns 

In [4]:
train_data["feature_category"] = train_data['ProductType']+"_"+train_data['Manufacturer']+"_"+train_data['Area Code']+"_"+train_data['Sourcing Channel']+"_"+train_data['Product Size']+"_"+train_data['Product Type']
train_feature_category = train_data["feature_category"].unique().tolist()
train_data["Date"] = pd.to_datetime(train_data["Month of Sourcing"])

test_data["feature_category"] = test_data['ProductType']+"_"+test_data['Manufacturer']+"_"+test_data['Area Code']+"_"+test_data['Sourcing Channel']+"_"+test_data['Product Size']+"_"+test_data['Product Type']
test_feature_category = test_data["feature_category"].unique().tolist()
test_data["Date"] = pd.to_datetime(test_data["Month of Sourcing"])


In [5]:
train_data.head()

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost,feature_category,Date
0,NTM3,X1,A28,WHOLESALE,Large,Powder,2021-05-01,10.158,NTM3_X1_A28_WHOLESALE_Large_Powder,2021-05-01
1,NTM2,X1,A9,DIRECT,Large,Powder,2020-10-01,134.281,NTM2_X1_A9_DIRECT_Large_Powder,2020-10-01
2,NTM3,X2,A20,DIRECT,Large,Powder,2020-12-01,12.456,NTM3_X2_A20_DIRECT_Large_Powder,2020-12-01
3,NTM3,X1,A18,WHOLESALE,Small,Powder,2021-02-01,107.220,NTM3_X1_A18_WHOLESALE_Small_Powder,2021-02-01
4,NTM2,X1,A28,DIRECT,Large,Liquid,2020-11-01,197.763,NTM2_X1_A28_DIRECT_Large_Liquid,2020-11-01


## Preprocessing Steps 
- For a specific key (label) I have considered mean value of sourcing cost if it has more than 1 sourcing cost for single date.
- Identified missing values and imputed them using pycaret's imputation method. - "Drift"
- Identified outlier using Qartiles (Q1,Q3,IQR) and outliers are treated accordingly.
## Modelling Steps
- Considered last 2 data points of training data as test data for model comparison.
- Considered time series models and trained on (n-2) records.
- Considered top 2 models based on MAPE metric.
- Combined top 2 models using blender functionality and have applied tuning functionality from pycaret library and again re trained on entire training data set.
- Forecasted the june month data using the final model for each of the label
## Preparing the dictionary with labels as keys
### Each Key will have information mentioned below
- missing_values -- (whether the training data of each label has any missing date between its minimum and maximum existing dates)
- data_statistics -- (statistical distribution metrics of training data for each label)
- train_data -- (contains training data after applying preprocessing steps - missing value treatment, outlier treatment also contains forecasted values for all the existing points using final model - yhat and if any data point is outlier treated)
- top_2_models -- (contains information about the selected top - 2 models based on MAPE metric)
- model_error_metrics -- (contains information of all the metric scores for all the models trained on training data)
- best_model_error_metrics -- (contains information of residual and error metric scores for the best model - top - 1 model )
- final_model -- (final model - which is first ensembled using top 2 models, fine tuned and at last retrained on entire training data)
- test_data -- (contains actual June data along with forecasted value using final model - yhat)


In [145]:
''' 
#### Hierarchy of label information ####

label_information 
    label
        missing_values,
        data_statistics
        train_data
        top_2_models
        model_error_metrics
        best_model_error_metrics
        final_model
        test_data

'''
label_information = {}
anomaly_feature_categories = []
selected_models = ['naive','arima','polytrend','auto_arima','exp_smooth','ets','tbats','prophet']


## Preprocessing, Outlier Treatment, Missing value treatment, Training on time series models, Ensembling Top 2 models, Forecasting for JUNE month (test data)  

In [1]:
for k in range(0,len(train_feature_category)):
    i = train_feature_category[k]
    print ("Started ==> ",i)
    label_information[i] = {}
    train_df = pd.DataFrame(train_data[train_data.feature_category==i].groupby("Date").apply(lambda x: round(x["Sourcing Cost"].mean(),2)),columns=["sourcing_cost"])
    train_df["Date"] = train_df.index
    
    min_date = train_df.Date.min()
    max_date = train_df.Date.max()
    date_df = pd.DataFrame(pd.date_range(start=min_date,end=max_date,freq="MS"),columns=["Date"])
    if train_df.shape[0] < date_df.shape[0]:
        label_information[i]["missing_values"] = ["yes",date_df.shape[0] - train_df.shape[0]]
        temp_df_1 = pd.merge(train_df.reset_index(drop=True),date_df,on="Date",how="outer").sort_values("Date")
        temp_df_1.index = temp_df_1.Date
        train_df = temp_df_1
    else:
        label_information[i]["missing_values"] = ["no",0]
        

    train_df["month_name"] = train_df.Date.apply(lambda x: x.month_name())
    train_df["month_num"] = train_df.Date.apply(lambda x: x.month)
    
    Q1 = train_df.sourcing_cost.quantile(0.25)
    Q3 = train_df.sourcing_cost.quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR
    train_df["Anomaly"] = 0
    train_df.loc[(train_df.sourcing_cost > upper_limit),"Anomaly"] = 1
    train_df.loc[(train_df.sourcing_cost < lower_limit),"Anomaly"] = 1
    
    train_df["source_outlier_treated"] = np.where(
                                                train_df["sourcing_cost"]>upper_limit,
                                                upper_limit,
                                                np.where(train_df["sourcing_cost"] < lower_limit, 
                                                lower_limit,
                                                train_df["sourcing_cost"])
                                                )
    if train_df[train_df.Anomaly==1].shape[0] > 0:
        anomaly_feature_categories.append(i)
        #print (i," '''Has Anomalies'''' ")
    
    
    label_information[i]["train_data"] = train_df
    
    temp_df = train_df.copy()
    
    ## Data Statistics
    try:
        df_setup = setup(data = temp_df[["source_outlier_treated","Date"]],index="Date", target = 'source_outlier_treated',fold=2,fh=3,numeric_imputation_target="drift")
    except Exception as e:
        print (str(e))
        df_setup = setup(data = temp_df[["source_outlier_treated","Date"]],index="Date", target = 'source_outlier_treated',fold=2,fh=2,numeric_imputation_target="drift")
        
    df_stats = check_stats()
    df_stats
    label_information[i]["data_statistics"] = df_stats
    
    # Model Training, top 2 models (MAPE based)
    df_best = compare_models(include=selected_models,sort="MAPE",n_select=2)
    df_models_metrics = pull()
    label_information[i]["top_2_models"] = df_best
    
    req_metrics = ["Model","MAE","RMSE","MAPE"]
    df_models_metrics_req = df_models_metrics[req_metrics]
    label_information[i]["model_error_metrics"] = df_models_metrics_req
    
    
    best_model_residuals_metrics = check_stats(estimator=df_best[0])
    best_model_residuals_metrics
    label_information[i]["best_model_error_metrics"] = best_model_residuals_metrics

    blender = blend_models(df_best)
    tuned_blender = tune_model(blender)
    final_model = finalize_model(tuned_blender)
    label_information[i]["final_model"] = final_model

    temp_t = plot_model(estimator=final_model,return_fig=False,return_data=True,plot="insample")
    y_hat = temp_t["overlay_data"].reset_index(drop=True)
    temp_df["y_hat"] = y_hat["EnsembleForecaster"].tolist()
    label_information[i]["train_data"] = temp_df
    
    
    temp_f = plot_model(final_model,plot="forecast",data_kwargs={"fh":3},return_data=True)
    
    
    test_df = pd.DataFrame(test_data[test_data.feature_category==i].groupby("Date").apply(lambda x: round(x["Sourcing Cost"].sum(),2)),columns=["sourcing_cost"])
    test_df["Date"] = test_df.index
    test_df["month_name"] = test_df.Date.apply(lambda x: x.month_name())
    test_df["month_num"] = test_df.Date.apply(lambda x: x.month)
    
    y_hat_future = temp_f["overlay_data"].reset_index(drop=True)
    test_df["y_hat"] = y_hat_future["EnsembleForecaster"].tolist()[:1]
    
    label_information[i]["test_data"] = test_df
    print ("Ended ==> ",i)


2

## Storing the label information as pickel file in local system

In [ ]:
import pickle
with open('label_information_mean.pickle', 'wb') as handle:
    pickle.dump(label_information, handle, protocol=pickle.HIGHEST_PROTOCOL)

    

'''
with open('label_information_mean.pickle', 'rb') as handle:
    b = pickle.load(handle)
'''


## Test data with forecasted value using final model for label - saving as csv into local system

In [158]:
test_forecasted_data = pd.concat([label_information[i]["test_data"].reset_index(drop=True) for i in train_feature_category],axis=0)
test_forecasted_data["label"] = train_feature_category
test_forecasted_data.to_csv("test_forecasted_data.csv",index=False)

## Train data with forecasted value using final model for label - saving as csv into local system

In [186]:
train_forecasted_data = [label_information[i]["train_data"].reset_index(drop=True) for i in train_feature_category]
label_train = []
for num,i in enumerate(train_feature_category):
    label_train = label_train + [i]*train_forecasted_data[num].shape[0]

train_forecasted_data = pd.concat(train_forecasted_data,axis=0)
train_forecasted_data["label"] = label_train
train_forecasted_data.to_csv("train_forecasted_data.csv",index=False)